http://www.gisdeveloper.co.kr/?p=8534

https://eunguru.tistory.com/237



tfjob 만드는 과정 => 지나치게 복잡하고 불편한게 현실인듯

1. jupyter에서 단순 학습 테스트 (이 jupyter 파일) (kubeflow jupyter)

2. .py 코드로 만듦 (개발 환경에서) (내 노트북에서)

3. dockerize 시킴 => container가 실행될때 파이썬 코드 실행시키게 만듦 (개발 환경에서) (내 노트북에서)

4. 이미지를 registry에 띄움 (개발 환경에서) (내 노트북에서) (여기서 ECR에 올리고, kubernetes에 secret 넣고 돌릴거 생각하면 힘드니까 개인 dockerhub에 올림)

5. tfjob yaml 파일 작성함 (개발 환경에서) (내 노트북에서)

6. kubectl apply -f 로실행함. (kubernetes terminal에서)

7. yaml파일을 jupyter에서 읽어서 pipeline으로 만듦 (kubeflow jupyter)

#총 3군데 환경을 계속 왔다갔다 해야 함.

#이게 중간에 yaml 파일만 고치면 바로 분산 학습이 될 줄 알았는데 잘 생각해보니까 학습 코드도 고쳐야됨 with strategy 넣어서
#이 과정도 주피터로 돌리고 난 다음 넘어가야지

In [2]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from tensorflow.keras.models import Model

/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
print(tf.__version__)
print(keras.__version__)

2.12.0
2.12.0


In [4]:
(x_train, y_train) , (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [6]:
print('num of machine: {}'.format(strategy.num_replicas_in_sync))

num of machine: 1


In [ ]:
#model = keras.models.Sequential([
#]

In [7]:
with strategy.scope():
    input_x = keras.layers.Input(shape=(28,28))

    x = Flatten()(input_x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_y = Dense(10, activation='softmax')(x)

    model = Model(inputs = input_x, outputs = output_y)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#0.5코어 주피터라 느리긴 하네
#짜놓고 보니까 valid set을 안넣었네?
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


2023-07-16 15:52:16.249541: W tensorflow/core/framework/dataset.cc:807] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


1875/1875 [==============================] - 44s 23ms/step - loss: 0.2968 - accuracy: 0.9145
Epoch 2/5
1131/1875 [=================>............] - ETA: 17s - loss: 0.1499 - accuracy: 0.9566

In [38]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 4s - loss: 0.0762 - accuracy: 0.9783 - 4s/epoch - 13ms/step


[0.0761503353714943, 0.9782999753952026]